In [ ]:
import syft as sy

In [ ]:
worker = sy.Worker.named("test-domain-1", reset=True)
domain_client = worker.root_client

In [ ]:
from jax import random
from flax import linen as nn
key = random.PRNGKey(42)

In [ ]:
train_data = random.uniform(key, shape=(4, 28, 28, 1))

In [ ]:
print("round(train_data.sum())", round(train_data.sum()))

In [ ]:
assert round(train_data.sum()) == 1602

In [ ]:
train = sy.ActionObject.from_obj(train_data)

In [ ]:
type(train.syft_action_data), train.id, train.shape

In [ ]:
_ = domain_client.api.services.action.set(train)

In [ ]:
class MLP(nn.Module):
    out_dims: int

    @nn.compact
    def __call__(self, x):
        x = x.reshape((x.shape[0], -1))
        x = nn.Dense(128)(x)
        x = nn.relu(x)
        x = nn.Dense(self.out_dims)(x)
        return x

model = MLP(out_dims=10)

In [ ]:
weights = model.init(key, train.syft_action_data)

In [ ]:
w = sy.ActionObject.from_obj(weights)

In [ ]:
type(w.syft_action_data), w.id

In [ ]:
_ = domain_client.api.services.action.set(w)

In [ ]:
@sy.syft_function(input_policy=sy.ExactMatch(weights=w.id, data=train.id),
                  output_policy=sy.SingleExecutionExactOutput())
def train_mlp(weights, data):
    from flax import linen as nn

    class MLP(nn.Module):
        out_dims: int

        @nn.compact
        def __call__(self, x):
            x = x.reshape((x.shape[0], -1))
            x = nn.Dense(128)(x)
            x = nn.relu(x)
            x = nn.Dense(self.out_dims)(x)
            return x

    model = MLP(out_dims=10)
    output = model.apply(weights, data)
    return output

In [ ]:
output = train_mlp(weights=weights, data=train_data)

In [ ]:
print("round(output.sum(), 2)", round(output.sum(), 2))

In [ ]:
assert round(output.sum(), 2) == -3.24

In [ ]:
request = domain_client.api.services.code.request_code_execution(train_mlp)
request

In [ ]:
request.approve()

In [ ]:
domain_client._api = None
_ = domain_client.api

In [ ]:
result = domain_client.api.services.code.train_mlp(weights=w.id, data=train.id)

In [ ]:
result

In [ ]:
print("round(float(result.sum()), 2)", round(float(result.sum()), 2))

In [ ]:
assert round(float(result.sum()), 2) == -3.24